In [ ]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import imageio.v3 as imageio
from IPython.display import clear_output
from render.CollectFrames import collect_saved_frames
matplotlib.use('Agg')

#### Function to calculate derivatives for RK4 algorithm

In [23]:
def calculate_derivatives(
        Y_vec: np.ndarray,
        masses: np.ndarray,
        lengths: np.ndarray,
        gravity: float):
    num_pendulums = len(masses)
    angles = Y_vec[0 : num_pendulums]
    angular_vels = Y_vec[num_pendulums : 2 * num_pendulums]
    # Derivative of angle is the angular velocity: dθᵢ/dt = ωᵢ
    d_angles_dt = angular_vels.copy()
    # The NxN inertia matrix in generalized coordinates
    inertia_matrix = np.zeros((num_pendulums, num_pendulums), dtype=float)
    for i in range(num_pendulums):
        for j in range(i, num_pendulums):
            # Mass bellow lower pendulum Σ[p=max(i,j) to N-1] mₚ
            mass_sum = np.sum(masses[max(i, j):])
            # cos(θᵢ - θⱼ) between the two pendulums
            cos_term = np.cos(angles[i] - angles[j])
            # Inertia value Mᵢⱼ = lᵢ lⱼ cos(θᵢ - θⱼ) Σ[p=max(i,j) to N-1] mₚ
            inertia_value = lengths[i] * lengths[j] * cos_term * mass_sum
            inertia_matrix[i, j] = inertia_value
            inertia_matrix[j, i] = inertia_value
    # The Nx1 vector of generalized forces (gravity + velocity effects)
    R_vec = np.zeros(num_pendulums)
    for k in range(num_pendulums):
        # Sum Σⱼ(...) velocity-dependent terms
        sum_vel_terms = 0.0
        for j in range(num_pendulums):
            # Mass bellow lower pendulum Σ[p=max(k,j) to N-1] mₚ
            mass_sum = np.sum(masses[max(k, j):])
            # sin(θₖ - θⱼ) between the two pendulums
            sin_term = np.sin(angles[k] - angles[j])
            # Full term: -lₖ lⱼ sin(θₖ - θⱼ) ωⱼ² Σ[p=max(k,j) to N-1] mₚ
            sum_vel_terms += -lengths[k] * lengths[j] * sin_term * (angular_vels[j]**2) * mass_sum
        # Mass bellow the current pendulum Σ[p=k to N-1] mₚ
        mass_sum = np.sum(masses[k:])
        # sin(θₖ) of the current pendulum's angle
        sin_term = np.sin(angles[k])
        # Gravitational torque Gₖ = -g lₖ sin(θₖ) Σ[p=k to N-1] mₚ
        grav_torque = -gravity * lengths[k] * sin_term * mass_sum
        # Combine terms Rₖ = Σⱼ(...) + Gₖ
        R_vec[k] = sum_vel_terms + grav_torque
    # Solves the linear system M * angular accelerations = R for angular accelerations
    angular_accels = np.linalg.solve(inertia_matrix, R_vec)
    return np.concatenate((d_angles_dt, angular_accels))

#### Function to iterate multi pendulum dynamic system using RK4

In [24]:
def iterate_multipendulum(
        angles: np.ndarray,
        angular_vels: np.ndarray,
        masses: np.ndarray,
        lengths: np.ndarray,
        num_iter = 1000,
        step = 0.001,
        gravity = 9.80665):
    num_pendulums = len(angles)
    Y_curr = np.concatenate((np.array(angles), np.array(angular_vels)))
    for iteration in range(num_iter):
        # k₁ ≈ Δt ⋅ F(Y_curr) - Derivative estimate at the beginning (t)
        k1 = step * calculate_derivatives(Y_curr, masses, lengths, gravity)
        # k₂ ≈ Δt ⋅ F(Y_curr + ½k₁) - Derivative estimate at midpoint (t + ½Δt) using k₁
        k2 = step * calculate_derivatives(Y_curr + 0.5 * k1, masses, lengths, gravity)
        # k₃ ≈ Δt ⋅ F(Y_curr + ½k₂) - Derivative estimate at 2nd midpoint (t + ½Δt) using k₂
        k3 = step * calculate_derivatives(Y_curr + 0.5 * k2, masses, lengths, gravity)
        # k₄ ≈ Δt ⋅ F(Y_curr + k₃) - Derivative estimate at the end (t + Δt) using k₃
        k4 = step * calculate_derivatives(Y_curr + k3, masses, lengths, gravity)
        # 4th Order Runge-Kutta (RK4) formula: Y(t+Δt) ≈ Y(t) + (k₁ + 2k₂ + 2k₃ + k₄) / 6
        Y_curr = Y_curr + (k1 + 2.0 * k2 + 2.0 * k3 + k4) / 6.0
        # Yield the pendulum step
        yield iteration, Y_curr[:num_pendulums]

#### Function to calculate pendulum end positions from calculated angles

In [25]:
def calculate_positions(center: tuple, angles: np.ndarray, lengths: np.ndarray):
    num_pendulums = len(angles)
    positions = np.zeros((num_pendulums + 1, 2))
    positions[0, :] = center
    for i in range(num_pendulums):
        # dx = lᵢ * sin(θᵢ), dy = -lᵢ * cos(θᵢ)
        positions[i + 1, 0] = positions[i, 0] + lengths[i] * np.sin(angles[i])
        positions[i + 1, 1] = positions[i, 1] - lengths[i] * np.cos(angles[i])
    return positions

#### Functions to render the multi pendulum frames

In [26]:
def create_base_frame(figure_size=(6, 6), dpi=90, bg_color=(0, 0, 0, 255)):
    width_px = int(figure_size[0] * dpi)
    height_px = int(figure_size[1] * dpi)
    initial_frame = np.full((height_px, width_px, 4), bg_color, dtype=np.uint8)
    return initial_frame

In [27]:
def alpha_composite_frames(foreground: np.ndarray, background: np.ndarray):
    fg_rgb = foreground[..., :3].astype(np.float32) / 255.0
    fg_a = foreground[..., 3:].astype(np.float32) / 255.0
    bg_rgb = background[..., :3].astype(np.float32) / 255.0
    bg_a = background[..., 3:].astype(np.float32) / 255.0
    out_a = fg_a + bg_a * (1.0 - fg_a)
    inv_out_a = np.divide(1.0, out_a, out=np.zeros_like(out_a), where=out_a > 1e-6)
    out_rgb = (fg_rgb * fg_a + bg_rgb * bg_a * (1.0 - fg_a)) * inv_out_a
    out_rgba = np.concatenate((out_rgb, out_a), axis=-1)
    out_rgba = np.clip(out_rgba, 0, 1)
    return (out_rgba * 255.0).astype(np.uint8)

In [28]:
def create_multipendulum_frame(
        prev_frame: np.ndarray,
        positions: np.ndarray,
        prev_positions: np.ndarray,
        iteration: int,
        colors: list,
        dpi = 90):
    num_pendulums = positions.shape[0] - 1
    height, width = prev_frame.shape[:2]
    figure_size = (width / dpi, height / dpi)
    plot_limit = 2.5
    lim = (-plot_limit, plot_limit)
    fig_temp1, ax_temp1 = plt.subplots(figsize=figure_size, dpi=dpi)
    ax_temp1.set_xlim(lim); ax_temp1.set_ylim(lim)
    ax_temp1.set_aspect('equal', adjustable='box'); ax_temp1.axis('off')
    fig_temp1.patch.set_alpha(0.0); ax_temp1.patch.set_alpha(0.0)
    if iteration > 0:
        for i in range(2, num_pendulums + 1):
            ax_temp1.plot(
                [prev_positions[i, 0], positions[i, 0]],
                [prev_positions[i, 1], positions[i, 1]],
                color=colors[i], alpha=0.6,
                linestyle='-', linewidth=1.5, 
                solid_capstyle='butt'
            )
    fig_temp1.canvas.draw()
    buffer1 = fig_temp1.canvas.buffer_rgba()
    new_tracers_layer = np.frombuffer(buffer1, dtype=np.uint8).reshape((height, width, 4))
    plt.close(fig_temp1)
    new_tracer_frame = alpha_composite_frames(foreground=new_tracers_layer, background=prev_frame)
    fig_temp2, ax_temp2 = plt.subplots(figsize=figure_size, dpi=dpi)
    ax_temp2.set_xlim(lim); ax_temp2.set_ylim(lim)
    ax_temp2.set_aspect('equal', adjustable='box'); ax_temp2.axis('off')
    fig_temp2.patch.set_alpha(0.0); ax_temp2.patch.set_alpha(0.0)
    for i in range(num_pendulums):
        ax_temp2.plot(
            [positions[i, 0], positions[i+1, 0]], 
            [positions[i, 1], positions[i+1, 1]], 
            color=colors[i+1], linewidth=2.5
        )
    ax_temp2.plot(positions[0, 0], positions[0, 1], 'o', markersize=5, color=colors[0])
    for i in range(1, num_pendulums + 1):
        ax_temp2.plot(positions[i, 0], positions[i, 1], 'o', markersize=8, color=colors[i])
    text = f'Iteration: {iteration}'
    ax_temp2.text(0, 1.0, text, transform=ax_temp2.transAxes, fontsize=11, color='white')
    fig_temp2.canvas.draw()
    buffer2 = fig_temp2.canvas.buffer_rgba()
    pendulum_layer = np.frombuffer(buffer2, dtype=np.uint8).reshape((height, width, 4))
    plt.close(fig_temp2)
    rendered_frame = alpha_composite_frames(foreground=pendulum_layer, background=new_tracer_frame)
    return new_tracer_frame.copy(), rendered_frame.copy()

#### Iterate the double pendulum system and collect frames

In [ ]:
folder_path = "gifs/pendulum/"
os.makedirs(folder_path, exist_ok=True)

In [31]:
center = (0.0, 0.0)
angles = np.array([np.pi / 2.1, np.pi / 1.9])
angular_vels = np.array([0.0, 0.0])
masses = np.array([1.0, 1.0])
lengths = np.array([1.0, 1.0])
colors = ['white', 'red', 'blue']
num_iter = 200000

In [32]:
positions = calculate_positions(center, angles, lengths)
prev_frame = create_base_frame()
prev_positions = positions
prev_render_positions = prev_positions
for it, angles_it in iterate_multipendulum(angles, angular_vels, masses, lengths, num_iter):
    positions = calculate_positions(center, angles_it, lengths)
    if it % 40 == 0:
        print(f"Double pendulum iteration: {it}")
        prev_frame, frame = create_multipendulum_frame(prev_frame, positions, prev_render_positions, it, colors)
        prev_render_positions = positions
        imageio.imwrite(os.path.join(folder_path, f"DoublePendulum{it:08d}.png"), frame)
    prev_positions = positions
clear_output()

In [33]:
double_pendulum_gif_filename = "DoublePendulum.gif"
collect_saved_frames(folder_path, double_pendulum_gif_filename, fps=25)

#### Visualize the double pendulum movement over time

![Double Pendulum](gifs/pendulum/DoublePendulum.gif)

#### Iterate double pendulum with slightly changed initial position and collect frames

In [35]:
center = (0.0, 0.0)
angles = np.array([np.pi / 2.1, np.pi / 1.9])
angular_vels = np.array([0.1, 0.0])
masses = np.array([1.0, 1.0])
lengths = np.array([1.0, 1.0])
colors = ['white', 'red', 'blue']
num_iter = 200000

In [36]:
positions = calculate_positions(center, angles, lengths)
prev_frame = create_base_frame()
prev_positions = positions
prev_render_positions = prev_positions
for it, angles_it in iterate_multipendulum(angles, angular_vels, masses, lengths, num_iter):
    positions = calculate_positions(center, angles_it, lengths)
    if it % 40 == 0:
        print(f"Double pendulum iteration: {it}")
        prev_frame, frame = create_multipendulum_frame(prev_frame, positions, prev_render_positions, it, colors)
        prev_render_positions = positions
        imageio.imwrite(os.path.join(folder_path, f"DoublePendulum2{it:08d}.png"), frame)
    prev_positions = positions
clear_output()

In [37]:
double_pendulum2_gif_filename = "DoublePendulum2.gif"
collect_saved_frames(folder_path, double_pendulum2_gif_filename, fps=25)

#### Visualize the double pendulum with slightly different parameters over time

![Double Pendulum 2](gifs/pendulum/DoublePendulum2.gif)

#### Iterate the multi pendulum system and collect frames

In [39]:
center = (0.0, 0.0)
angles = np.array([np.pi / 2.1, np.pi / 1.9005, np.pi / 1.8, np.pi / 1.7])
angular_vels = np.array([0.0, 0.0, 0.0, 0.0])
masses = np.array([0.5, 0.5, 0.5, 0.5])
lengths = np.array([0.5, 0.5, 0.5, 0.5])
colors = ['white', 'red', 'blue', 'green', 'purple']
num_iter = 200000

In [40]:
positions = calculate_positions(center, angles, lengths)
prev_frame = create_base_frame()
prev_positions = positions
prev_render_positions = prev_positions
for it, angles_it in iterate_multipendulum(angles, angular_vels, masses, lengths, num_iter):
    positions = calculate_positions(center, angles_it, lengths)
    if it % 40 == 0:
        print(f"Multi pendulum iteration: {it}")
        prev_frame, frame = create_multipendulum_frame(prev_frame, positions, prev_render_positions, it, colors)
        prev_render_positions = positions
        imageio.imwrite(os.path.join(folder_path, f"MultiPendulum{it:08d}.png"), frame)
    prev_positions = positions
clear_output()

In [41]:
multi_pendulum_gif_filename = "MultiPendulum.gif"
collect_saved_frames(folder_path, multi_pendulum_gif_filename, fps=25)

#### Visualize the multi pendulum movement over time

![Multi Pendulum](gifs/pendulum/MultiPendulum.gif)